In [ ]:
import openai
from typing import List
from pydantic import BaseModel
import json

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="sk-")
openai.api_key = "sk-"

In [ ]:
# --- Pydantic Models ---
class Claim(BaseModel):
    text: str

class PremiseUnlabeled(BaseModel):
    text: str

class ADUClassificationOutput(BaseModel):
    claims: List[Claim]
    premises: List[PremiseUnlabeled]

In [ ]:
paragraph = "Remote work improves employee productivity. People working remotely often report fewer distractions and better focus. EVs have zero tailpipe emissions. Most EVs are charged with electricity generated from fossil fuels. AI-generated images mimic style but lack original thought. 3D-printed organs with the help of AI could reduce transplant wait times."

In [ ]:
def classify_adu(text: str) -> ADUClassificationOutput:
    prompt = f"""
You are an expert in argument mining. Your job is to classify each sentence in the text as either a CLAIM or a PREMISE.

Instructions:
- You will receive a text.
- Each sentence in the text is either a claim or a premise.
- Return a valid JSON response in this format:
{{
  "claims": [{{"text": "..."}}],
  "premises": [{{"text": "..."}}]
}}

Paragraph:
\"\"\"
{text}
\"\"\"
Only return the JSON. Do not include explanations.
"""
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        response_format={"type": "json_object"}
    )
    return ADUClassificationOutput.model_validate_json(response.choices[0].message.content)

In [ ]:
# Get the result from the function
result = classify_adu(paragraph)

# Print the validated and formatted output
print(result.model_dump_json(indent=2))  # preferred in Pydantic v2+

{
  "claims": [
    {
      "text": "Remote work improves employee productivity."
    },
    {
      "text": "EVs have zero tailpipe emissions."
    },
    {
      "text": "AI-generated images mimic style but lack original thought."
    },
    {
      "text": "3D-printed organs with the help of AI could reduce transplant wait times."
    }
  ],
  "premises": [
    {
      "text": "People working remotely often report fewer distractions and better focus."
    },
    {
      "text": "Most EVs are charged with electricity generated from fossil fuels."
    }
  ]
}
